# Imports

In [1]:
import numpy as np
import sys
import os
#%load_ext autoreload

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#from src.utils.audio_to_spectrograms import *
from src.models.yamnet_train import *

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    path_prefix = ''
else:
    path_prefix = '..'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

2025-12-06 18:48:05.344957: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-06 18:48:05.345362: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-06 18:48:05.399361: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-06 18:48:19.282278: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

Using: cpu


In [ ]:
# uncomment to retrain
# manual train test split (stratified)
"""np.random.seed(0)
data = pickle.load(open('data/processed/yamnet/spectrograms_train.pkl', 'rb'))
train_size = 0.8
train_idx = []
for label in np.unique(data['event_label']):
    choices = np.where(data['event_label'] == label)[0]
    train_idx.append(np.sort(np.random.choice(choices, size = int(np.round(len(choices)*train_size)), replace = False)))
train_idx = np.sort(np.concatenate(train_idx))
len(train_idx)

solver = Solver(epochs = 5, train_idx=train_idx, path_prefix = path_prefix, device = device)
solver.train()"""

2001

In [4]:
test_path = Path("data") / "processed" / "yamnet" / "spectrograms_test.pkl"
test_data = pickle.load(open(test_path, "rb"))
filepaths = [os.path.join(DETECTION_TEST_PATH, file) for file in test_data['files']]
events = run_yamnet(filepaths)

2025-12-06 18:48:42.424 | INFO     | src.models.yamnet_train:run_yamnet:351 - Loading from checkpoint checkpoints/yamnet_detector.pth...
2025-12-06 18:48:46.950 | INFO     | src.models.yamnet_train:load_model:165 - Loaded model weights from checkpoints/yamnet_detector.pth


Pre-computing features for 500 files...


Testing...: 100%|██████████| 500/500 [00:07<00:00, 71.23it/s]


In [10]:
events['test_snipped_scene_0101.wav']

[{'file': 'test_snipped_scene_0101.wav',
  'event_onset': 6.24,
  'event_offset': 12.0}]

# Cut audio

In [ ]:
import soundfile as sf
import shutil
#from src.main import cut_events_from_audio

In [29]:
def cut_events_from_audio(extracted_audio_path, events_list, data_path=DETECTION_TEST_PATH):

    if not os.path.exists(extracted_audio_path):
        os.makedirs(extracted_audio_path)
        logger.info(f"Created directory: {extracted_audio_path}")
    else:
        # Clean the filepath as every prediction is new
        shutil.rmtree(extracted_audio_path)
        os.makedirs(extracted_audio_path)
        logger.info(f"Cleaned directory: {extracted_audio_path}")

    for filename, events in tqdm(events_list.items()):
        filename = Path(data_path, filename)
        for event in events:
            start_sec = event["event_onset"]
            end_sec = event["event_offset"]

            audio_array, sr = sf.read(filename)
            base_name = Path(filename).stem

            # Cut the wav file and save it to the extracted_audio_path
            start_sample = int(start_sec * sr)
            end_sample = int(end_sec * sr)

            # Slice the numpy array
            sliced_audio = audio_array[start_sample:end_sample]

            # Construct new filename
            new_filename = f"{base_name}_{start_sec:.2f}_{end_sec:.2f}.wav"
            output_path = extracted_audio_path / new_filename

            # Write the new file
            sf.write(output_path, sliced_audio, sr)

In [30]:
cut_events_from_audio((Path("data") / "processed" / "yamnet" / "extracted_audio"), events)

2025-12-06 19:03:18.547 | INFO     | __main__:cut_events_from_audio:10 - Cleaned directory: data/processed/yamnet/extracted_audio
  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [00:59<00:00,  8.37it/s]
